# Test Multi-Band Model

In [1]:
from ASTROMER.core.multiband_astromer import get_MBAstromer

## Test Model Instantiation

In [2]:
model_names = ["macho", "atlas"]

In [3]:
model = get_MBAstromer(model_names)

## Test Processing Attention Vectors

In [4]:
model.summary()

Model: "Multi-Band ASTROMER"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 200, 1)]             0         []                            
                                                                                                  
 length (InputLayer)         [(None, 200, 1)]             0         []                            
                                                                                                  
 mask_in (InputLayer)        [(None, 200, 1)]             0         []                            
                                                                                                  
 times (InputLayer)          [(None, 200, 1)]             0         []                            
                                                                                

In [5]:
import numpy as np

In [6]:
data = [
    np.array(
        [
            [5200, 0.3, 0.2],
            [5300, 0.5, 0.1],
            [5400, 0.2, 0.3],
        ]
    ),
    np.array(
        [
            [4200, 0.3, 0.1],
            [4300, 0.6, 0.3],
        ]
    )
]

In [7]:
import tensorflow as tf

reduce_mean_layer = model.get_layer("reduce_mean")
intermediate_model = tf.keras.Model(inputs=model.inputs, outputs=reduce_mean_layer.output)

In [8]:
from ASTROMER.core.data import load_numpy

dataset = load_numpy(data, ids=None, labels=None, batch_size=1, sampling=False, shuffle=False, max_obs=200)

att, lens, ids = [],[],[]
for batch in dataset:
    emb = intermediate_model(batch)

    sizes = tf.reduce_sum(1-batch['mask_in'], 1)
    sizes = tf.cast(sizes, tf.int32)

    att.append(emb)
    lens.append(sizes)
    ids.append([str(b.numpy().decode('utf8')) for b in batch['id']])

concatenate = True
if concatenate:
    att = np.concatenate(att, 0)
    lens = np.concatenate(lens, 0)
    ids = np.concatenate(ids, 0)

    final_att = []
    for oid in np.unique(ids):
        indices = np.where(ids == oid)
        foo = np.concatenate(att[indices], 0)
        goo = np.sum(lens[indices])
        final_att.append(foo[:goo])
    print(final_att)
print(att)

[INFO] no masking


/Users/gchiong/repos/astromer-python-lib/env/lib/python3.10/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['output', 'mask_out', 'label', 'id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


[array([[ 1.38576597e-01,  1.33697093e-01,  1.12318136e-01,
         4.09023583e-01, -4.88118589e-01,  5.14294803e-02,
         1.97446555e-01, -7.49544621e-01,  1.44698524e+00,
         1.27530074e+00,  1.57682151e-01,  1.85547069e-01,
         1.41805246e-01,  3.36992472e-01,  1.27942711e-01,
         1.17173418e-02,  5.85050061e-02,  4.68000919e-02,
         2.55586714e-01,  8.28913748e-02,  1.39199167e-01,
         6.96817636e-02,  1.23008624e-01,  7.65585899e-02,
         8.95563316e+00,  3.33201468e-01, -3.50352144e+00,
        -6.00112975e-02,  1.29522145e-01,  1.26417965e-01,
         1.19224273e-01,  4.47909355e-01, -8.30441952e-01,
         1.67386279e-01, -6.20823383e-01,  1.02857664e-01,
         8.17909911e-02,  1.91465929e-01, -8.37410069e+00,
        -3.43518168e-01,  1.29173234e-01,  1.42576978e-01,
         1.59335539e-01, -2.29722166e+00,  1.26459527e+00,
        -1.91788304e+00,  1.37397543e-01, -1.01250365e-01,
         9.89243537e-02,  1.44086868e-01,  1.24656111e-